In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras

In [179]:
df = pd.read_csv('WineQT.csv')
df = df.dropna()
df = df.drop(columns=['Id'])
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [180]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,0.996730,3.311015,0.657708,10.442111,5.657043
std,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.001925,0.156664,0.170399,1.082196,0.805824
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,0.995570,3.205000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,0.996680,3.310000,0.620000,10.200000,6.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,0.997845,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [181]:
# print column data types
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [182]:
print(df['quality'].value_counts())

quality
5    483
6    462
7    143
4     33
8     16
3      6
Name: count, dtype: int64


In [183]:
# convert Quality to a multi-class label
df['quality'] = pd.cut(df['quality'], bins=[0, 5, 6, 10], labels=['low', 'medium', 'high'])

In [184]:
print(df['quality'].value_counts())

quality
low       522
medium    462
high      159
Name: count, dtype: int64


In [185]:
train, test = train_test_split(df, test_size=0.2)
X_train, y_train = train.drop('quality', axis=1), train['quality']
X_test, y_test = test.drop('quality', axis=1), test['quality']

In [186]:
# create a model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(11,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

In [187]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [188]:
# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Fit the model
model.fit(X_train, y_train_encoded, epochs=200)


Epoch 1/200
29/29 [==============================] - 2s 3ms/step - loss: 2.6249 - accuracy: 0.4212
Epoch 2/200
29/29 [==============================] - 0s 3ms/step - loss: 1.0481 - accuracy: 0.5077
Epoch 3/200
29/29 [==============================] - 0s 3ms/step - loss: 0.9759 - accuracy: 0.5175
Epoch 4/200
29/29 [==============================] - 0s 3ms/step - loss: 1.0373 - accuracy: 0.4923
Epoch 5/200
29/29 [==============================] - 0s 4ms/step - loss: 0.9419 - accuracy: 0.5197
Epoch 6/200
29/29 [==============================] - 0s 4ms/step - loss: 0.9586 - accuracy: 0.5088
Epoch 7/200
29/29 [==============================] - 0s 3ms/step - loss: 0.9288 - accuracy: 0.5460
Epoch 8/200
29/29 [==============================] - 0s 4ms/step - loss: 0.9189 - accuracy: 0.5449
Epoch 9/200
29/29 [==============================] - 0s 4ms/step - loss: 0.9098 - accuracy: 0.5350
Epoch 10/200
29/29 [==============================] - 0s 4ms/step - loss: 0.9211 - accuracy: 0.5514
Epoch 11/

In [189]:
# Evaluate the model using a confusion matrix and classification report
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test_encoded, y_pred)
print(classification_report(y_test_encoded, y_pred))
print(cm)

8/8 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.70      0.42      0.53        45
           1       0.67      0.82      0.74       107
           2       0.49      0.45      0.47        77

    accuracy                           0.62       229
   macro avg       0.62      0.57      0.58       229
weighted avg       0.62      0.62      0.61       229

[[19  7 19]
 [ 2 88 17]
 [ 6 36 35]]
